# 第19章 马尔可夫链蒙特卡罗法    
     
    19.1 蒙特卡罗法    
        19.1.1 随机抽样    
        19.1.2 数学期望估计    
        19.1.3 积分计算    
    19.2 马尔可夫链    
        19.2.1 基本定义    
        19.2.2 离散状态马尔可夫链    
        19.2.3 连续状态马尔可夫链    
        19.2.4 马尔可夫链的性质    
    19.3 马尔可夫链蒙特卡罗法    
        19.3.1 基本想法    
        19.3.2 基本步骤    
        19.3.3 马尔可夫链蒙特卡罗法与统计学习    
    19.4 Metropolis-Hastings 算法    
        19.4.1 基本原理    
        19.4.2 Metropolis-Hastings 算法    
        19.4.3 单分量 Metropolis-Hastings 算法    
    19.5 吉布斯抽样    
        19.5.1 基本原理    
        19.5.2 吉布斯抽样算法    
        19.5.3 抽样计算  
    

In [1]:
import random
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

蒙特卡罗法（Monte Carlo method），也称为统计模拟方法（statistical simulation method），是通过从概率模型的随机抽样进行近似数值计算的方法。
马尔可夫链蒙特卡罗法（Markov Chain Monte Carlo, MCMC）,则是以马尔可夫链（Markov chain）为概率模型的蒙特卡罗法。
马尔可夫链蒙特卡罗法构建一个马尔可夫链，使其平稳分布就是要进行抽样的分布，首先基于该马尔可夫链进行随机游走，产生样本的序列，之后使用该平稳分布的样本进行近似数值计算。

Metropolis - Hastings 算法足最基本的马尔可夫链蒙特卡罗法，Metropolis等人在1953年提出原始的算法，Hastings在1970年对之加以推广，形成了现在的形式。
吉布斯抽样（Gibbs sampling）是更简单、使用更广泛的马尔可夫链蒙特卡罗法，1984年由S.Geman和D.Geman提出。

马尔可夫链蒙特卡罗法被应用于概率分布的估计、定积分的近似计算、最优化问题的近似求解等问题，特别是被应用于统计学习中概率模型的学习与推理，是重要的统计学习计算方法。

## 19.4 Metropolis-Hastings 算法

Metropolis-Hastings算法是马尔可夫链蒙特卡罗法的代表算法。

###     19.4.1 基本原理    

#### 1. 马尔科夫链

假设要抽样的概率分布为 **Metropolis-Hastings算法** 采用转移核为 $p(x, x')$的马尔可夫链：

$$ p(x,x')=q(x,x')\alpha(x,x') $$

其中 $q(x,x')$ 和 $\alpha(x,x')$ 分别称为建议分布（proposal distribution）和接受分布（acceptance distribution）。

建议分布 $q(x,x')$ 是另一个马尔可夫链的转移核，并且 $q(x,x')$ 是不可约的，即其概 率值恒不为 $0$，同时是一个容易抽样的分布。
接受分布是

$$\alpha\left(x, x^{\prime}\right)=\min \left\{1, \frac{p\left(x^{\prime}\right) q\left(x^{\prime}, x\right)}{p(x) q\left(x, x^{\prime}\right)}\right\}$$

这时，转移核可以写成

$$p\left(x, x^{\prime}\right)=\left\{\begin{array}{ll}
{q\left(x, x^{\prime}\right),} & {p\left(x^{\prime}\right) q\left(x^{\prime}, x\right) \geqslant p(x) q\left(x, x^{\prime}\right)} \\
{q\left(x^{\prime}, x\right) \frac{p\left(x^{\prime}\right)}{p(x)},} & {p\left(x^{\prime}\right) q\left(x^{\prime}, x\right)<p(x) q\left(x, x^{\prime}\right)}
\end{array}\right.$$

转移核为 $p(x,x')$ 的马尔可夫链上的随机游走以以下方式进行。
如果在时刻 $(t- 1)$ 处于状态 $x$，即 $x_{t-1}=x$，则先按建议分布 $q(x,x')$ 抽样产生一个候选状态 $x'$，然后按照接受分布 $\alpha(x,x')$ 抽样决定是否接受状态 $x'$。
以概率 $\alpha(x,x')$ 接受 $x'$，决定时刻 $t$ 转移到状态 $x'$，而以概率 $ 1-\alpha(x, x')$ 拒绝 $x'$，决定时刻 $t$ 仍停留在状态 $x$。
具体地，从区间 $(0,1)$ 上的均匀分布中抽取一个随机数 $u$，决定时刻 $t$ 的状态。

$$ x_{t}=\left\{\begin{array}{ll}
{x^{\prime},} & {u \leqslant \alpha\left(x, x^{\prime}\right)} \\
{x,} & {u>\alpha\left(x, x^{\prime}\right)}
\end{array}\right. $$

可以证明，转移核为 $p(x,x')$ 的马尔可夫链是可逆马尔可夫链（满足遍历定理），其平稳分布就是 $p(x)$ ，即要抽样的目标分布。
也就是说，这是马尔可夫链蒙特卡罗法的一个具体实现。

**由转移核构成的马尔可夫链是可逆的，即**

$$p(x)p(x,x')=p(x')p(x',x)$$

**并且 $p(x)$ 是该马尔可夫链的平稳分布。**

#### 2. 建议分布

建议分布 $q(x,x')$ 有多种可能的形式，这里介绍两种常用形式。

**第一种形式**

假设建议分布是对称的，即对任意的 $x$ 和 $x'$ 有

$$ q(x,x')=q(x',x) $$

这样的建议分布称为 Metropolis 选择，也是 Metropolis-Hastings 算法最初采用的建议分布。
这时，接受分布 $\alpha(x,x')$ 简化为

$$ \alpha\left(x, x^{\prime}\right)=\min \left\{1, \frac{p\left(x^{\prime}\right)}{p(x)}\right\} $$

Metropolis 选择的一个特例是 $q(x,x')$ 取条件概率分布 $p(x'|x)$，定义为多元正态分布，其均值是 $x$，其协方差矩阵是常数矩阵。

Metropolis 选择的另一个特例是令 $ q(x,x') = q(|x-x'|)$，这时算法称为随机游走 Metropolis 算法。
例如，

$$ q\left(x, x^{\prime}\right) \propto \exp \left(-\frac{\left(x^{\prime}-x\right)^{2}}{2}\right) $$

Metropolis 选择的特点是当 $x'$ 与 $x$ 接近时，$q(x,x')$ 的概率值高，否则 $q(x,x')$ 的概率值低。
状态转移在附近点的可能性更大。

**第二种形式**

独立抽样。
假设 $q(x,x')$ 与当前状态 $x$ 无关，即 $q(x,x') = q(x')$。
建议分布的计算按照 $q(x')$ 独立抽样进行。
此时，接受分布 $\alpha(x,x')$ 可以写成

$$ \alpha\left(x, x^{\prime}\right)=\min \left\{1, \frac{w\left(x^{\prime}\right)}{w(x)}\right\} $$

其中 $w(x')=p(x')/q(x'), w(x)=p(x)/q(x)$

独立抽样实现简单，但可能收敛速度慢，通常选择接近目标分布 $p(x)$ 的分布作为建议分布 $q(x)$。


#### 3. 满条件分布

###     19.4.2 Metropolis-Hastings 算法

输入：抽样的目标分布的密度函数 $p(x)$，函数 $f(x)$；

输出：$p(x)$ 的随机样本 $x_{m+1}, x_{m+2}, \cdots, x_n$，函数样本均值 $f_{mn}$；

参数：收敛步数 $m$ ，迭代步数 $n$。

1. 任意选择一个初始值$x_0$
2. 对 $i=l,2,\cdots,n$ 循环执行

    a. 设状态 $x_{i-1}=x$，按照建议分布 $q(x,x')$ 随机抽取一个候选状态 $x'$。

    b. 计算接受概率
    
    $$ \alpha\left(x, x^{\prime}\right)=\min \left\{1, \frac{p\left(x^{\prime}\right) q\left(x^{\prime}, x\right)}{p(x) q\left(x, x^{\prime}\right)}\right\} $$
    
    c. 从区间 $(0,1)$ 中按均匀分布随机抽取一个数 $u$。
    若 $u \leq \alpha(x,x')$，则状态 $x_i=x'$；否则，状态 $x'=x$。

3. 得到样本集合 $\{x_{m+1}, x_{m+2}, \cdots, x_n\}$，计算
    
    $$ f_{m n}=\frac{1}{n-m} \sum_{i=m+1}^{n} f\left(x_{i}\right) $$

###     19.4.3 单分量 Metropolis-Hastings 算法    

## 19.5 吉布斯抽样    

###     19.5.1 基本原理    

###     19.5.2 吉布斯抽样算法    

###     19.5.3 抽样计算  
